In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

Matplotlib is building the font cache; this may take a moment.


In [ ]:
# Read the vocabulary of words for names
with open('names.txt', 'r') as f:
    words = f.read().splitlines()

In [ ]:
len(words)